# Using MySQL Connector

In [7]:
import mysql.connector as sql
import datetime
import csv
import pandas as pd
import os

connection = sql.connect(
  host="localhost",
  user="root",
  password="Ahmed@1470963_"
)
cursor = connection.cursor()

In [ ]:
connection = sql.connect(
  host="localhost",
  user="root",
  password="Ahmed@1470963_"
)
cursor = connection.cursor()

# CSV file path
csv_file_path = './data/orders.csv'
df = pd.read_csv(csv_file_path)

# Create schema (database) if it doesn't exist
schema_name = 'instacart'
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {schema_name}")

# Use the instacart schema
cursor.execute(f"USE instacart")

# Get column names and types from the dataframe
columns = df.columns
column_definitions = []

# Define a simple mapping for pandas data types to MySQL data types
dtype_mapping = {
    'int64': 'INT',
    'float64': 'FLOAT',
    'object': 'VARCHAR(255)',
    'bool': 'BOOLEAN',
    'datetime64': 'DATETIME',
    'timedelta64': 'TIME'
}

# Create column definitions for SQL table creation
for column in columns:
    dtype = str(df[column].dtype)
    sql_type = dtype_mapping.get(dtype, 'VARCHAR(255)')  # Default to VARCHAR if type not found
    column_definitions.append(f"{column} {sql_type}")

# Generate the CREATE TABLE SQL statement
# Extract the table name from the file name (without extension)
table_name = os.path.splitext(os.path.basename(csv_file_path))[0]

create_table_sql = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        {', '.join(column_definitions)}
    )
"""
print(create_table_sql)  # Print the CREATE TABLE statement to verify

# Execute the CREATE TABLE statement
cursor.execute(create_table_sql)

# Prepare the bulk insert statement
insert_sql = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(columns))})"

# Perform the bulk insert
cursor.executemany(insert_sql, df.to_records(index=False).tolist())

# Commit the transaction
connection.commit()

# Close the connection
cursor.close()
connection.close()

In [6]:
import os
import pandas as pd
import mysql.connector
from mysql.connector import OperationalError

# CSV file path
csv_file_path = './data/products.csv'

# Extract the table name from the file name (without extension)
table_name = os.path.splitext(os.path.basename(csv_file_path))[0]

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)
# Fill NaN values with 0
df = df.fillna(0)

#df = df.head(100000)

def connect_to_db():
    return mysql.connector.connect(
  host="localhost",
  user="root",
  password="Ahmed@1470963_"
)
try:
    connection = connect_to_db()
    cursor = connection.cursor()

    # Set global timeout settings
    cursor.execute("SET GLOBAL wait_timeout = 28800")
    cursor.execute("SET GLOBAL interactive_timeout = 28800")
    cursor.execute("SET GLOBAL max_allowed_packet = 67108864")  # 64 MB

    # Create schema (database) if it doesn't exist
    schema_name = 'instacart'
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {schema_name}")
    cursor.execute(f"USE {schema_name}")

    # Get column names and types from the dataframe
    columns = df.columns
    column_definitions = []

    # Define a simple mapping for pandas data types to MySQL data types
    dtype_mapping = {
        'int64': 'INT',
        'float64': 'FLOAT',
        'object': 'VARCHAR(255)',
        'bool': 'BOOLEAN',
        'datetime64': 'DATETIME',
        'timedelta64': 'TIME'
    }

    # Create column definitions for SQL table creation
    for column in columns:
        dtype = str(df[column].dtype)
        sql_type = dtype_mapping.get(dtype, 'VARCHAR(255)')  # Default to VARCHAR if type not found
        column_definitions.append(f"{column} {sql_type}")

    # Generate the CREATE TABLE SQL statement using the dynamic table name
    create_table_sql = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            {', '.join(column_definitions)}
        )
    """
    cursor.execute(create_table_sql)

    # Prepare the bulk insert statement
    insert_sql = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(['%s'] * len(columns))})"

    # Perform batch insertion
    batch_size = 1000
    records = df.to_records(index=False).tolist()

    for i in range(0, len(records), batch_size):
        batch = records[i:i + batch_size]
        cursor.executemany(insert_sql, batch)
        connection.commit()  # Commit after each batch

except OperationalError as e:
    if e.errno == 2013:
        print("Connection lost. Reconnecting...")
        connection = connect_to_db()
        cursor = connection.cursor()
        cursor.execute(f"USE {schema_name}")
        # Retry your operations here
finally:
    if cursor:
        cursor.close()
    if connection:
        connection.close()


In [30]:
df.shape

(3421083, 7)

In [5]:
import mysql.connector as sql
import datetime

cnx = sql.connect(
  host="localhost",
  user="root",
  password="PASSWORD",
  database="employees"
)
cursor = cnx.cursor()

query = ("SELECT first_name, last_name, hire_date FROM employees "
         "WHERE hire_date BETWEEN %s AND %s")

hire_start = datetime.date(1999, 1, 1)
hire_end = datetime.date(1999, 12, 31)

cursor.execute(query, (hire_start, hire_end))

for (first_name, last_name, hire_date) in cursor:
  print("{}, {} was hired on {:%d %b %Y}".format(
    last_name, first_name, hire_date))

cursor.close()
cnx.close()

Haddadi, Lillian was hired on 30 Apr 1999
Piveteau, Hironoby was hired on 23 Mar 1999
DuCasse, Dietrich was hired on 30 Mar 1999
Tokunaga, Aimee was hired on 28 Oct 1999
Meriste, Neven was hired on 17 Jul 1999
Rossi, Samphel was hired on 10 Jan 1999
Merey, JoAnne was hired on 06 Nov 1999
Magliocco, Malu was hired on 23 Aug 1999
Pettis, Chaosheng was hired on 17 Feb 1999
Guenter, Kensei was hired on 04 Jan 1999
Rosaz, Zvonko was hired on 01 Feb 1999
Kushner, Billie was hired on 24 Jan 1999
Casperson, Percy was hired on 08 Oct 1999
Erni, Mooi was hired on 01 Oct 1999
Marzano, Shawna was hired on 04 Jan 1999
Mukaidono, Mayumi was hired on 22 Jan 1999
Ranst, Jingling was hired on 19 Jun 1999
Bazzichi, Yuuichi was hired on 01 Jul 1999
Rullman, Tetsurou was hired on 06 Mar 1999
Attimonelli, Junichi was hired on 30 Jan 1999
Siepmann, Adil was hired on 31 Dec 1999
Llado, Mabry was hired on 24 Jan 1999
Kragelund, Pragnesh was hired on 29 May 1999
Undy, Yonghoan was hired on 29 Sep 1999
Poulakid

In [6]:
import mysql.connector as sql
import datetime

cnx = sql.connect(
  host="localhost",
  user="root",
  password="PASSWORD",
  database="employees"
)
cursor = cnx.cursor()

query = ("SELECT first_name, last_name, hire_date FROM employees "
         "WHERE hire_date BETWEEN %s AND %s")

hire_start = datetime.date(1999, 1, 1)
hire_end = datetime.date(1999, 12, 31)

cursor.execute(query, (hire_start, hire_end))

cursor.fetchone()

('Lillian', 'Haddadi', datetime.date(1999, 4, 30))

In [7]:
cursor.fetchall()

[('Hironoby', 'Piveteau', datetime.date(1999, 3, 23)),
 ('Dietrich', 'DuCasse', datetime.date(1999, 3, 30)),
 ('Aimee', 'Tokunaga', datetime.date(1999, 10, 28)),
 ('Neven', 'Meriste', datetime.date(1999, 7, 17)),
 ('Samphel', 'Rossi', datetime.date(1999, 1, 10)),
 ('JoAnne', 'Merey', datetime.date(1999, 11, 6)),
 ('Malu', 'Magliocco', datetime.date(1999, 8, 23)),
 ('Chaosheng', 'Pettis', datetime.date(1999, 2, 17)),
 ('Kensei', 'Guenter', datetime.date(1999, 1, 4)),
 ('Zvonko', 'Rosaz', datetime.date(1999, 2, 1)),
 ('Billie', 'Kushner', datetime.date(1999, 1, 24)),
 ('Percy', 'Casperson', datetime.date(1999, 10, 8)),
 ('Mooi', 'Erni', datetime.date(1999, 10, 1)),
 ('Shawna', 'Marzano', datetime.date(1999, 1, 4)),
 ('Mayumi', 'Mukaidono', datetime.date(1999, 1, 22)),
 ('Jingling', 'Ranst', datetime.date(1999, 6, 19)),
 ('Yuuichi', 'Bazzichi', datetime.date(1999, 7, 1)),
 ('Tetsurou', 'Rullman', datetime.date(1999, 3, 6)),
 ('Junichi', 'Attimonelli', datetime.date(1999, 1, 30)),
 ('Adil'

In [8]:
cursor.close()
cnx.close()

# Using Pandas

In [9]:
import pandas as pd

In [10]:
cnx = sql.connect(
  host="localhost",
  user="root",
  password="PASSWORD",
  database="employees"
)

In [13]:
employees = pd.read_sql("SELECT * FROM employees", cnx)

C:\Users\ahosny\AppData\Local\Temp\ipykernel_32392\1225823882.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  employees = pd.read_sql("SELECT * FROM employees", cnx)


In [14]:
employees.head()

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12
